### Dataset

In [46]:
import pandas as pd
from auto_ts import auto_timeseries
import dill
import talib
import numpy as np
import warnings

warnings.filterwarnings('ignore')

from sklearn.model_selection import TimeSeriesSplit
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout, Flatten, Conv1D, MaxPooling1D, BatchNormalization
from keras.regularizers import l2
from keras.optimizers import Adam, RMSprop, SGD
from scikeras.wrappers import KerasRegressor
from keras.callbacks import EarlyStopping
from skopt import BayesSearchCV
from sklearn.preprocessing import LabelEncoder
import tensorflow.keras.backend as K
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OneHotEncoder
from keras.losses import categorical_crossentropy
from sklearn.metrics import log_loss
from keras.utils import to_categorical
from lightgbm import LGBMClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import StratifiedKFold

In [47]:
# Evito que ciertas columnas se transformen a notacion cientifica en las predicciones
pd.set_option('display.float_format', lambda x: '%.2f' % x)
pd.set_option('display.max_columns', None)

In [48]:
columns = [
    # 'Open_time',
    # 'Close',
    'Open',
    'High',
    'Low',
    'SMA_20',
    'EMA_20',
    'Upper_Band',
    'Middle_Band',
    'Lower_Band',
    'RSI',
    'MACD',
    'Signal',
    'ADX',
    'SlowK',
    'SlowD',
    'CCI',
    'ATR',
]

### Armado y entrenamiento de un clasificador a partir de los datos originales

#### Modelo Light GBM

In [49]:
complete_dataset = pd.read_csv('/Users/mmarchetta/Desktop/Tesis-2024/data-visualization/final_dataset.csv') 
classifier_dataset = complete_dataset[columns]
# classifier_dataset['Open_time'] = pd.to_datetime(classifier_dataset['Open_time'])
classifier_dataset['Tendencia'] = complete_dataset['Tendencia']

clasifier_validation = classifier_dataset[-10:]
classifier_dataset = classifier_dataset[:-10]

In [50]:
display(classifier_dataset.tail())

,Open,High,Low,SMA_20,EMA_20,Upper_Band,Middle_Band,Lower_Band,RSI,MACD,Signal,ADX,SlowK,SlowD,CCI,ATR,Tendencia
941,6.93,7.00,6.70,7.43,7.45,9.08,7.43,5.77,38.83,-0.48,-0.53,33.89,45.65,47.62,-11.66,0.60,Lateral
942,6.86,6.95,6.71,7.34,7.38,8.94,7.34,5.74,37.81,-0.48,-0.52,34.06,47.38,47.30,-14.45,0.57,Lateral
943,6.76,6.87,6.51,7.24,7.33,8.73,7.24,5.76,38.57,-0.47,-0.51,34.51,47.54,46.86,-44.81,0.56,Lateral
944,6.81,6.95,6.69,7.13,7.27,8.38,7.13,5.88,37.66,-0.46,-0.50,34.64,45.17,46.70,-30.25,0.53,Lateral
945,6.73,6.83,6.47,7.03,7.20,8.08,7.03,5.97,36.02,-0.46,-0.49,35.10,34.26,42.32,-73.58,0.52,Bajista


In [51]:
classifier_dataset.shape

(946, 17)

In [52]:
X = classifier_dataset.drop(columns=["Tendencia"])
y = classifier_dataset["Tendencia"]

y = y.to_numpy().reshape(-1, 1)
onehot_encoder = OneHotEncoder(sparse=False)
y_one_hot = onehot_encoder.fit_transform(y)


In [53]:
display(y_one_hot)

array([[0., 0., 1.],
       [0., 1., 0.],
       [1., 0., 0.],
       ...,
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 1., 0.]])

In [54]:
classifier = LGBMClassifier(objective='multiclass', num_class=3, metric='multi_logloss', verbosity=-1)

# Definir el espacio de búsqueda de hiperparámetros
param_space = {
    'num_leaves': (10, 500),
    'learning_rate': (0.001, 0.5), 
    'n_estimators': (50, 500),
    'min_child_samples': (5, 200),
    'subsample': (0.5, 1.0),  
    'colsample_bytree': (0.5, 1.0),
    'n_iter': (50, 500),
    'reg_alpha': (0.0, 1.0),
    'reg_lambda': (0.0, 1.0),
    'objective': ['multiclass'],
    'metric': ['multi_logloss', 'multi_error'],
    'importance_type': ['gain', 'split'],
    'boosting_type': ['gbdt', 'dart', 'rf'],
}
# Definir función de pérdida para la búsqueda bayesiana
def multi_log_loss(estimator, X_test, y_test):
    y_pred = estimator.predict_proba(X_test)
    loss = log_loss(y_test, y_pred)
    return loss

# Definir validación cruzada temporal
cv = TimeSeriesSplit(n_splits=5).split(X)
# stratified_cv = StratifiedKFold(n_splits=10, shuffle=False)

# Realizar la búsqueda bayesiana de hiperparámetros
bayes_search = BayesSearchCV(
    classifier, 
    param_space, 
    scoring=multi_log_loss, 
    cv=cv,
    # cv=TimeSeriesSplit(n_splits=10),
    # cv=stratified_cv,
    # n_iter=100,
    verbose=0,
    n_jobs=-1
)

# Realizar la búsqueda bayesiana
bayes_result = bayes_search.fit(X, y)

/Users/mmarchetta/Desktop/Tesis-2024/myenv/lib/python3.8/site-packages/sklearn/preprocessing/_label.py:99: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/mmarchetta/Desktop/Tesis-2024/myenv/lib/python3.8/site-packages/sklearn/preprocessing/_label.py:99: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/mmarchetta/Desktop/Tesis-2024/myenv/lib/python3.8/site-packages/sklearn/preprocessing/_label.py:99: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/mmarchetta/Desktop/Tesis-2024/myenv/lib/python3.8/site-packages/sklearn/preprocessing/_label.py:99: DataC

In [55]:
# Show best results
print("Best score:", bayes_result.best_score_)
print("Best parameters:", bayes_result.best_params_)

# Entrenar el modelo con los mejores hiperparámetros
best_model = bayes_result.best_estimator_
best_model.fit(X, y)

Best score: 3.12533472531326
Best parameters: OrderedDict([('boosting_type', 'gbdt'), ('colsample_bytree', 0.5), ('importance_type', 'split'), ('learning_rate', 0.5), ('metric', 'multi_error'), ('min_child_samples', 5), ('n_estimators', 500), ('n_iter', 500), ('num_leaves', 10), ('objective', 'multiclass'), ('reg_alpha', 0.0), ('reg_lambda', 0.0), ('subsample', 0.5)])


LGBMClassifier(colsample_bytree=0.5, learning_rate=0.5, metric='multi_error',
               min_child_samples=5, n_estimators=500, n_iter=500, num_class=3,
               num_leaves=10, objective='multiclass', subsample=0.5,
               verbosity=-1)

In [56]:
import json

# Obtener los hiperparámetros y puntajes de los 5 mejores modelos
top_n_models = 5
best_params_list = []
best_scores_list = []

for i in range(min(top_n_models, len(bayes_search.cv_results_['params']))):
    best_params_list.append(bayes_search.cv_results_['params'][i])
    best_scores_list.append(bayes_search.cv_results_['mean_test_score'][i])

# Guardar los hiperparámetros de los 5 mejores modelos en un archivo JSON
with open('gbm_classifier/top_5_hyperparameters.json', 'w') as f:
    json.dump({'best_params': best_params_list, 'best_scores': best_scores_list}, f)

# O imprimir los hiperparámetros
print("Top 5 mejores modelos:")
for i in range(len(best_params_list)):
    print("Modelo", i+1)
    print("Hiperparámetros:", best_params_list[i])
    print("Puntaje:", best_scores_list[i])


Top 5 mejores modelos:
Modelo 1
Hiperparámetros: OrderedDict([('boosting_type', 'rf'), ('colsample_bytree', 0.9399080803509805), ('importance_type', 'split'), ('learning_rate', 0.27688521526808013), ('metric', 'multi_error'), ('min_child_samples', 108), ('n_estimators', 58), ('n_iter', 326), ('num_leaves', 431), ('objective', 'multiclass'), ('reg_alpha', 0.8736239844125129), ('reg_lambda', 0.9264039086229062), ('subsample', 0.9719228099702548)])
Puntaje: 1.0594460768729372
Modelo 2
Hiperparámetros: OrderedDict([('boosting_type', 'rf'), ('colsample_bytree', 0.8897168188089601), ('importance_type', 'gain'), ('learning_rate', 0.45590432105799894), ('metric', 'multi_error'), ('min_child_samples', 54), ('n_estimators', 347), ('n_iter', 317), ('num_leaves', 17), ('objective', 'multiclass'), ('reg_alpha', 0.2410739124423092), ('reg_lambda', 0.2224412625248931), ('subsample', 0.8038068054600633)])
Puntaje: 1.0352565796223576
Modelo 3
Hiperparámetros: OrderedDict([('boosting_type', 'dart'), ('c

#### Armado del ensamble

In [57]:
# Crear una lista para almacenar los modelos individuales
individual_models = []

# Entrenar los modelos individuales
for params_str in best_params_list:
    model = LGBMClassifier(verbosity=-1, **params_str)
    model.fit(X, y)
    individual_models.append(model)

voting_model = VotingClassifier(estimators=[('model_'+str(i), model) for i, model in enumerate(individual_models)], voting='hard', verbose=0)#voting='soft', verbose=0)
voting_model.fit(X, y)

with open('gbm_classifier/gbm_boosting_classifier.pkl', 'wb') as f:
    dill.dump(voting_model, f)

#### Clasificacion con el ensamble sobre las redicciones de los modelos generativos

In [58]:
with open('gbm_classifier/gbm_boosting_classifier.pkl', 'rb') as f:
    voting_model = dill.load(f)

In [59]:
voting_model.fit(X, y)

VotingClassifier(estimators=[('model_0',
                              LGBMClassifier(boosting_type='rf',
                                             colsample_bytree=0.9399080803509805,
                                             learning_rate=0.27688521526808013,
                                             metric='multi_error',
                                             min_child_samples=108,
                                             n_estimators=58, n_iter=326,
                                             num_leaves=431,
                                             objective='multiclass',
                                             reg_alpha=0.8736239844125129,
                                             reg_lambda=0.9264039086229062,
                                             subsample=0.9719228099702548,
                                             verbosity=...
                                             verbosity=-1)),
                             ('model_4',
                              LGBMClassifier(colsample_bytree=0.6882173514883003,
                                             importance_type='gain',
                                             learning_rate=0.07090903602425687,
                                             metric='multi_logloss',
                                             min_child_samples=142,
                                             n_estimators=401, n_iter=450,
                                             num_leaves=88,
                                             objective='multiclass',
                                             reg_alpha=0.31280907223440774,
                                             reg_lambda=0.8768967157736816,
                                             subsample=0.9700410842828294,
                                             verbosity=-1))],
                 verbose=0)

##### Datos originales:

In [60]:
display(clasifier_validation)

,Open,High,Low,SMA_20,EMA_20,Upper_Band,Middle_Band,Lower_Band,RSI,MACD,Signal,ADX,SlowK,SlowD,CCI,ATR,Tendencia
946,6.59,6.67,6.04,6.93,7.13,7.80,6.93,6.05,34.18,-0.46,-0.49,36.09,27.85,35.76,-133.97,0.53,Bajista
947,6.42,6.93,6.13,6.85,7.11,7.41,6.85,6.29,43.30,-0.43,-0.48,36.14,34.84,32.32,-60.17,0.55,Alcista
948,6.90,7.41,6.69,6.85,7.12,7.42,6.85,6.29,49.27,-0.36,-0.45,34.85,53.51,38.73,61.80,0.56,Alcista
949,7.27,7.39,7.00,6.90,7.14,7.44,6.90,6.35,48.86,-0.31,-0.42,33.66,71.22,53.19,72.86,0.55,Lateral
950,7.24,7.28,7.11,6.91,7.13,7.46,6.91,6.36,46.98,-0.28,-0.39,32.55,76.04,66.93,57.12,0.52,Bajista
951,7.12,7.40,7.01,6.94,7.15,7.51,6.94,6.37,50.00,-0.23,-0.36,31.17,76.67,74.65,71.03,0.51,Alcista
952,7.30,7.47,7.06,6.96,7.15,7.53,6.96,6.39,47.10,-0.21,-0.33,29.68,74.61,75.77,71.86,0.50,Bajista
953,7.12,7.29,6.98,6.98,7.13,7.52,6.98,6.44,45.10,-0.20,-0.30,28.48,73.00,74.76,45.52,0.49,Bajista
954,6.99,7.19,6.89,6.99,7.12,7.52,6.99,6.46,44.94,-0.19,-0.28,27.59,67.67,71.76,28.65,0.48,Lateral
955,6.98,7.09,6.78,7.01,7.11,7.52,7.01,6.50,46.32,-0.17,-0.26,27.02,67.19,69.28,12.46,0.47,Lateral


In [61]:
n_clases = 3 

validation_predictions = voting_model.predict(clasifier_validation[columns])
display(validation_predictions)

array(['Bajista', 'Alcista', 'Alcista', 'Alcista', 'Bajista', 'Alcista',
       'Bajista', 'Bajista', 'Bajista', 'Bajista'], dtype=object)

##### Datos generados por auto ml con Auto-TS:

In [64]:
auto_ts_feature_predictor_df = pd.read_csv('auto_ts_models/result.csv')
features_auto_ts = auto_ts_feature_predictor_df[columns]

predictions_ts = voting_model.predict(features_auto_ts)

display(predictions_ts)

array(['Bajista', 'Alcista', 'Alcista', 'Alcista', 'Bajista', 'Bajista',
       'Bajista', 'Bajista', 'Bajista', 'Bajista'], dtype=object)

##### Datos generados por auto ml con prophet:

In [63]:
auto_ml_prophet_df = pd.read_csv('auto_timeseries_models_prophet/predicciones.csv')
auto_mp_prophet_predictions = voting_model.predict(auto_ml_prophet_df[columns].drop(columns=["Open_time"]))
display(auto_mp_prophet_predictions)

KeyError: "['MACD', 'Signal', 'ADX', 'SlowK', 'SlowD', 'CCI', 'ATR'] not in index"

##### Datos generados por auto ml con stats:

In [ ]:
auto_ml_stats_df = pd.read_csv('auto_timeseries_models/predicciones.csv')
auto_mp_stats_predictions = voting_model.predict(auto_ml_stats_df[columns].drop(columns=["Open_time"]))

display(auto_mp_stats_predictions)

array(['Bajista', 'Bajista', 'Bajista', 'Bajista', 'Bajista', 'Bajista',
       'Bajista', 'Bajista', 'Bajista', 'Bajista'], dtype=object)

##### Datos generados por auto ml con modelos clasicos:

In [ ]:
auto_ml_df = pd.read_csv('h2o_models/predicciones.csv')
auto_mp_predictions = voting_model.predict(auto_ml_df[columns].drop(columns=["Open_time"]))
display(auto_mp_predictions)

array(['Bajista', 'Bajista', 'Bajista', 'Bajista', 'Bajista', 'Bajista',
       'Bajista', 'Bajista', 'Bajista', 'Bajista'], dtype=object)

##### Datos generados por auto ml con skforecast:

In [ ]:
skforecast_df = pd.read_csv('skforecast/predicciones.csv')
skforecast_predictions = voting_model.predict(skforecast_df[columns[1:]])
display(skforecast_predictions)

array(['Bajista', 'Lateral', 'Alcista', 'Alcista', 'Alcista', 'Alcista',
       'Alcista', 'Alcista', 'Alcista', 'Alcista'], dtype=object)